In [109]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [18]:
data = pd.read_csv('../merged_final_with_revisions.csv')

/srv/paws/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,9,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
data.head()

,Unnamed: 0,id,uk_page_title,uk_page_id,uk_translations_count,uk_incoming_links,uk_langlinks_count,outcoming_links,uk_first_edit,uk_last_edit,uk_revisions_count,uk_minor_revisions_count,uk_deleted_revisions,outcoming_links_translated,en_page_title_clear,en_page_id,en_page_title,en_first_edit
0,0,NaN,!_(альбом_С.К.А.Й.),829565.0,0.0,31.0,NaN,40.0,2010-10-14 09:59:01,2017-04-23 15:40:11,43.0,13.0,0.0,NaN,NaN,NaN,NaN,NaN
1,1,4540205.0,!_(альбом),2112530.0,0.0,NaN,12.0,12.0,2016-01-29 23:11:32,2018-03-06 11:57:03,10.0,1.0,0.0,NaN,!_(The_Dismemberment_Plan_album),3243047.0,!_(The_Dismemberment_Plan_album),2005-11-26 01:28:16
2,2,404000.0,!_(значення),1366003.0,0.0,1.0,16.0,32.0,2012-09-08 09:20:52,2016-05-18 09:49:18,7.0,5.0,0.0,NaN,!_(disambiguation),64486.0,!_(disambiguation),2002-07-25 17:36:03
3,3,353153.0,!!_(значення),2141483.0,0.0,NaN,17.0,18.0,2016-03-11 16:18:49,2016-06-01 20:48:15,5.0,0.0,0.0,NaN,!!,3632887.0,!!,2006-01-06 03:37:09
4,4,371.0,!!!,425480.0,24.0,8.0,25.0,36.0,2009-03-28 07:57:13,2018-04-05 10:46:42,28.0,20.0,0.0,36.0,!!!,600744.0,!!!,2004-04-16 19:00:58


In [61]:
data_clean = data.drop(columns=['Unnamed: 0', 'en_page_title_clear', 'id', 'uk_page_id', 'en_page_id', 'en_page_title'])

In [62]:
data_clean['uk_incoming_links'] = data_clean['uk_incoming_links'].fillna(0)
data_clean['uk_langlinks_count'] = data_clean['uk_langlinks_count'].fillna(0)
data_clean['outcoming_links_translated'] = data_clean['outcoming_links_translated'].fillna(0)

In [63]:
# the bool variable for getting know if the article has already been translated
data_clean['is_translated'] = (1-data_clean['en_first_edit'].isnull().astype(int))

In [64]:
# from str to numpy.datetime64
data_clean['uk_last_edit'] =  pd.to_datetime(data_clean['uk_last_edit'], unit='ns')
data_clean['uk_first_edit'] =  pd.to_datetime(data_clean['uk_first_edit'], unit='ns')
data_clean['en_first_edit'] =  pd.to_datetime(data_clean['en_first_edit'], unit='ns')

In [65]:
import datetime
# set date for filling na in translated date. it could be a month or more 
current = np.datetime64(datetime.datetime.now())

In [86]:
data_clean['en_first_edit'] =  data_clean['en_first_edit'].fillna(current)
# difference in days
data_clean['days_to_translate'] = ((data_clean['en_first_edit'] -  data_clean['uk_first_edit']).astype(int))/(10**9 * 60 * 60 * 24)

In [87]:
data_clean.head()

,uk_page_title,uk_translations_count,uk_incoming_links,uk_langlinks_count,outcoming_links,uk_first_edit,uk_last_edit,uk_revisions_count,uk_minor_revisions_count,uk_deleted_revisions,outcoming_links_translated,en_first_edit,is_translated,days_to_translate
0,!_(альбом_С.К.А.Й.),0.0,31.0,0.0,40.0,2010-10-14 09:59:01,2017-04-23 15:40:11,43.0,13.0,0.0,0.0,2018-07-22 05:40:14.778358,0,2837.820298
1,!_(альбом),0.0,0.0,12.0,12.0,2016-01-29 23:11:32,2018-03-06 11:57:03,10.0,1.0,0.0,0.0,2005-11-26 01:28:16.000000,1,-3716.905046
2,!_(значення),0.0,1.0,16.0,32.0,2012-09-08 09:20:52,2016-05-18 09:49:18,7.0,5.0,0.0,0.0,2002-07-25 17:36:03.000000,1,-3697.656123
3,!!_(значення),0.0,0.0,17.0,18.0,2016-03-11 16:18:49,2016-06-01 20:48:15,5.0,0.0,0.0,0.0,2006-01-06 03:37:09.000000,1,-3717.528935
4,!!!,24.0,8.0,25.0,36.0,2009-03-28 07:57:13,2018-04-05 10:46:42,28.0,20.0,0.0,36.0,2004-04-16 19:00:58.000000,1,-1806.539062


In [103]:
data_clean['translated_from_en'] = ((data_clean['en_first_edit'] < data_clean['uk_first_edit'] )& data_clean['is_translated'] == 1).astype(int)
data_clean['translated_from_uk'] = ((data_clean['en_first_edit'] > data_clean['uk_first_edit'] )& data_clean['is_translated'] == 1).astype(int)

In [105]:
data_clean_final = data_clean.drop(columns=['uk_first_edit', 'uk_last_edit', 'en_first_edit'])

In [106]:
data_clean_final.head()

,uk_page_title,uk_translations_count,uk_incoming_links,uk_langlinks_count,outcoming_links,uk_revisions_count,uk_minor_revisions_count,uk_deleted_revisions,outcoming_links_translated,is_translated,days_to_translate,translated_from_uk,translated_from_en
0,!_(альбом_С.К.А.Й.),0.0,31.0,0.0,40.0,43.0,13.0,0.0,0.0,0,2837.820298,0,0
1,!_(альбом),0.0,0.0,12.0,12.0,10.0,1.0,0.0,0.0,1,-3716.905046,0,1
2,!_(значення),0.0,1.0,16.0,32.0,7.0,5.0,0.0,0.0,1,-3697.656123,0,1
3,!!_(значення),0.0,0.0,17.0,18.0,5.0,0.0,0.0,0.0,1,-3717.528935,0,1
4,!!!,24.0,8.0,25.0,36.0,28.0,20.0,0.0,36.0,1,-1806.539062,0,1


In [108]:
data_clean_final.to_csv('train_test_data.csv')

In [110]:
data_clean.to_csv('full_data.csv')